In [1]:
import torch
import torch.nn as nn

In [2]:
# previous ones are using batch gradient desecent while training
# total dataset is forward passed and so on

In [3]:
# using Dataset and Dataloader for Mini Batch Gradient Descent

In [4]:
from sklearn.datasets import make_classification

In [5]:
X, y = make_classification(

    n_samples = 10,
    n_features = 2,
    n_informative = 2,
    n_redundant = 0,
    n_classes = 2,
    random_state = 42
    )

In [6]:
X

array([[ 1.06833894, -0.97007347],
       [-1.14021544, -0.83879234],
       [-2.8953973 ,  1.97686236],
       [-0.72063436, -0.96059253],
       [-1.96287438, -0.99225135],
       [-0.9382051 , -0.54304815],
       [ 1.72725924, -1.18582677],
       [ 1.77736657,  1.51157598],
       [ 1.89969252,  0.83444483],
       [-0.58723065, -1.97171753]])

In [7]:
y

array([1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [10]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

C:\Users\sauvi\AppData\Local\Temp\ipykernel_11320\3998235742.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)
C:\Users\sauvi\AppData\Local\Temp\ipykernel_11320\3998235742.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.long)


In [12]:
from torch.utils.data import Dataset, DataLoader

In [14]:
class CustomDataset(Dataset):

    def __init__(self, features, labels):

        self.features = features
        self.labels = labels

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

In [15]:
dataset = CustomDataset(X, y)

In [16]:
len(dataset)

10

In [17]:
dataset[0]

(tensor([ 1.0683, -0.9701]), tensor([1, 0]))

In [18]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [19]:
for batch_features, batch_labels in dataloader:

    print(batch_features)
    print(batch_labels)

tensor([[ 1.7774,  1.5116],
        [-0.7206, -0.9606]])
tensor([[1, 1],
        [0, 0]])
tensor([[ 1.7273, -1.1858],
        [-1.9629, -0.9923]])
tensor([[ 1, -1],
        [-1,  0]])
tensor([[-0.9382, -0.5430],
        [ 1.0683, -0.9701]])
tensor([[0, 0],
        [1, 0]])
tensor([[-1.1402, -0.8388],
        [ 1.8997,  0.8344]])
tensor([[-1,  0],
        [ 1,  0]])
tensor([[-2.8954,  1.9769],
        [-0.5872, -1.9717]])
tensor([[-2,  1],
        [ 0, -1]])


# Breast Cancer classification using DataLoader

In [20]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [21]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [22]:
df.shape

(569, 33)

In [23]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [24]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [26]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [27]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [28]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [29]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [30]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):

  def __init__(self, features, labels):

    self.features = features
    self.labels = labels

  def __len__(self):

    return len(self.features)

  def __getitem__(self, idx):

    return self.features[idx], self.labels[idx]

In [31]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

In [32]:
train_dataset[10]

(tensor([ 0.0235,  0.0746,  0.2324, -0.0827, -1.3495,  2.4009,  2.8286,  0.7813,
         -0.3761,  2.2192, -0.1605,  0.4769,  0.3000, -0.2583, -0.5682,  2.8709,
          3.6615,  1.8399,  0.6428,  3.6744, -0.2176, -0.3348, -0.0238, -0.3132,
         -1.9238,  1.1141,  2.0183,  0.5623, -0.8014,  1.4171]),
 tensor(0.))

In [33]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [34]:
import torch.nn as nn


class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

In [35]:
learning_rate = 0.1
epochs = 25

In [36]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loss function
loss_function = nn.BCELoss()

In [37]:

# define loop
for epoch in range(epochs):

  for batch_features, batch_labels in train_loader:

    # forward pass
    y_pred = model(batch_features)

    # loss calculate
    loss = loss_function(y_pred, batch_labels.view(-1,1))

    # clear gradients
    optimizer.zero_grad()

    # backward pass
    loss.backward()

    # parameters update
    optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.16348843276500702
Epoch: 2, Loss: 0.05390555411577225
Epoch: 3, Loss: 0.2360346019268036
Epoch: 4, Loss: 0.1100052073597908
Epoch: 5, Loss: 0.09092338383197784
Epoch: 6, Loss: 0.25750160217285156
Epoch: 7, Loss: 0.14517727494239807
Epoch: 8, Loss: 0.03452868014574051
Epoch: 9, Loss: 0.12957444787025452
Epoch: 10, Loss: 0.018870536237955093
Epoch: 11, Loss: 0.1784507781267166
Epoch: 12, Loss: 0.024182865396142006
Epoch: 13, Loss: 0.3299121856689453
Epoch: 14, Loss: 0.2652687728404999
Epoch: 15, Loss: 0.26056355237960815
Epoch: 16, Loss: 0.01926003210246563
Epoch: 17, Loss: 0.14479069411754608
Epoch: 18, Loss: 0.025949345901608467
Epoch: 19, Loss: 0.016444247215986252
Epoch: 20, Loss: 0.06284749507904053
Epoch: 21, Loss: 0.06676442176103592
Epoch: 22, Loss: 0.12027966231107712
Epoch: 23, Loss: 0.012792268767952919
Epoch: 24, Loss: 0.014034858904778957
Epoch: 25, Loss: 0.10581016540527344


In [38]:
# Model evaluation using test_loader
model.eval()  # Set the model to evaluation mode
accuracy_list = []

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        # Forward pass
        y_pred = model(batch_features)
        y_pred = (y_pred > 0.8).float()  # Convert probabilities to binary predictions

        # Calculate accuracy for the current batch
        batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
        accuracy_list.append(batch_accuracy)

# Calculate overall accuracy
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy: {overall_accuracy:.4f}')

Accuracy: 0.9844
